In [1]:
with open('token_vk.txt', 'r') as file_object:
    token_vk = file_object.read().strip()

with open('token_yandex.txt', 'r') as file_object:
    token_yandex = file_object.read().strip()
    
version = '5.126'

In [13]:
import requests
import os
import time
import tqdm
import tqdm.notebook
import sys


class VkUser:
    url = 'https://api.vk.com/method/'

    def __init__(self, token_vk, version, user_id=None):
        self.user_id = user_id
        self.token_vk = token_vk
        self.version = version
        self.params = {
            'user_id': self.user_id,
            'access_token': self.token_vk,
            'v': self.version     
        }
        self.owner_id = requests.get(self.url + 'users.get', self.params).json()['response'][0]['id']
        
        
        
        
    def get_photos(self, user_id=None):
        if user_id is None:
            user_id = self.owner_id
        photos_url = self.url + 'photos.get'
        photos_params = {
            'owner_id': self.user_id,
            'album_id': 'profile',
            'extended': 1

        }
        res = requests.get(photos_url, params = {**self.params, **photos_params})
          

        res = res.json()['response']['items']
        
        likes = []
        for photo in res:
            like = photo['likes']['count']
            likes.append(like)
        
        likes = [x for x in likes if likes.count(x) == 1]
    
       
        urllist = []
        photos = []
        for photo in res:
            urls = photo['sizes'][-1]['url']
            urllist.append([urls])
            file_name = photo['likes']['count']
            file_size =  photo['sizes'][-1]['type']
            file_date = photo['date']
            if file_name not in likes:
                file_name = str(file_name) +'_'+ str(file_date) + '.jpg'
            else:
                file_name = str(file_name) + '.jpg'
            photos.append({'file_name': file_name, 'size': file_size})
        
        
        return urllist, photos
    
    def save_photos(self, user_id=None):
        if user_id is None:
            user_id = self.owner_id
            
        photos_urls = user.get_photos()[0]
        file_names = [file['file_name'] for file in user.get_photos()[1]]
        print(file_names)
        
        i = 0
        for urls in photos_urls:
            print(urls)
            file_response = requests.get(urls[0])
            file_name = file_names[i]
            print(file_name)
            with open('photos/%s' % file_name, 'wb') as file:
                file.write(file_response.content)
            i += 1
            
    def upload_to_yandex(self, user_id=None, folder_name='vk_photos', photos_number=5):
        if user_id is None:
            user_id = self.owner_id
        if photos_number == '':
            photos_number = 5
            
        uploader = YaUploader(token_yandex)        
        user = VkUser(token_vk, version, user_id)
        
        uploader.folder(folder_name)
        
        photos_urls = user.get_photos()[0]
        file_names = [file['file_name'] for file in user.get_photos()[1]]
        
        photos_urls = photos_urls[0:int(photos_number)]
        
        i = 0
        
        pbar = tqdm.notebook.tqdm(photos_urls, desc='Uploading files')
            
        for urls in pbar:
            file_response = requests.get(urls[0])
            file_name = file_names[i]
            result = uploader.upload(folder_name, file_name, file_response.content)
            i += 1
            time.sleep(0.1)




In [3]:
class YaUploader:
    def __init__(self, token_yandex: str):
        self.token_yandex = token_yandex

    def upload(self, folder_name, file_name, file):
        """Метод загружает файл file на яндекс диск"""
        TOKEN = token_yandex
        HEADERS = {
            'Authorization': f'OAuth {TOKEN}'
        }
        
        
        response = requests.get(
            'https://cloud-api.yandex.net/v1/disk/resources/upload',
            params={
                'path': str(folder_name + '/' + file_name),
                'overwrite': True
            },
            headers=HEADERS
        )

        response.raise_for_status()

        href = response.json()['href']
        

        upload_response = requests.put(href, files={'file': file})
        upload_response.raise_for_status()
                
        return print(f'Файл {file_name} успешно загружен!')
    
    def folder(self, folder_name):
        """Метод создает папку folder_name на яндекс диск"""
        
        TOKEN = token_yandex
        HEADERS = {
            'Authorization': f'OAuth {TOKEN}'
        }
        
        response = requests.put(
            'https://cloud-api.yandex.net/v1/disk/resources',
            params={
                'path': str(folder_name)
                
            },
             headers=HEADERS
        )
        
        response.raise_for_status()
        
        


In [17]:
def main():
    user_input = input('Введите id пользователя: ')
    if user_input == '':
        user_id = None
    else:
        user_id = user_input
        
    user_input = input('Введите Yandex токен: ')
    #     token_yandex = user_input
    user_input = input('Введите имя папки: ')
    folder_name = user_input
    user_input = input('Сколько фотографий сохранить на диск: ')
    if user_input == 0:
        sys.exit()
    photos_number = user_input
    
        
    with open('token_vk.txt', 'r') as file_object:
        token_vk = file_object.read().strip()

    version = '5.126'
        
    user = VkUser(token_vk, version, user_id)
    res = user.get_photos()
    
    if folder_name == '':
        user.upload_to_yandex()
    else:
        user.upload_to_yandex(user_id, folder_name, photos_number)
    
    
    return(res)

In [18]:
main()

Введите id пользователя: 
Введите Yandex токен: 
Введите имя папки: new
Сколько фотографий сохранить на диск: 2


Файл 12.jpg успешно загружен!
Файл 3.jpg успешно загружен!



([['https://sun9-28.userapi.com/impf/c850732/v850732336/16fa43/3b7pxN3vzmI.jpg?size=400x400&quality=96&proxy=1&sign=e43d16ae62020287b2e5ae9073d3f878&c_uniq_tag=EzmDlYlDwGUQNchZD8cUjUTNJKc7bCwdLiOZrTNSNEI&type=album'],
  ['https://sun9-64.userapi.com/impg/oFiwQBOqvXfWUtxWvqxcMyvSeMIZhiK3ki9Ssw/1J6DiX6JDr8.jpg?size=800x800&quality=96&proxy=1&sign=bba662d0ca0bcff8ed8e4ab5e5b5266e&c_uniq_tag=5RbEpQ4RVpWrd6BxOtyz2740QME47ovKwoKiZdZ8JXM&type=album'],
  ['https://sun9-45.userapi.com/impg/9ArHk5QgSaCV92n0W_V0AiB9qcVaaF2dSBGhKQ/LgiiF_lk2DM.jpg?size=600x800&quality=96&proxy=1&sign=a44427270a7788fdb76b0e2589d51cb5&c_uniq_tag=LCwjbFfAaExFnUonYDYouVvBZaT4vXeg0_OmwM7c39Q&type=album'],
  ['https://sun9-45.userapi.com/impg/9ArHk5QgSaCV92n0W_V0AiB9qcVaaF2dSBGhKQ/LgiiF_lk2DM.jpg?size=600x800&quality=96&proxy=1&sign=a44427270a7788fdb76b0e2589d51cb5&c_uniq_tag=LCwjbFfAaExFnUonYDYouVvBZaT4vXeg0_OmwM7c39Q&type=album'],
  ['https://sun9-48.userapi.com/impg/hZBImqkCXbzy3VGIhn30Jj4P3RWurerkaZZSnw/oDCgWwpSe34.j